# Grading curve

This course has a curve due to [SIPA policy](https://bulletin.columbia.edu/sipa/academic-policies/grading-system-academic-progress/):

> Grades submitted for SIPA core courses must have an average GPA between 3.2 and 3.4, with the goal being 3.3. Courses with enrollments over 35 are also recommended to follow this rule.

## Getting your estimated course grade

1. [Open {{lms_name}}.]({{lms_url}})
1. Go to `Grades`.
1. View the [current grade](https://community.canvaslms.com/t5/Student-Guide/How-do-I-view-my-grades-in-a-current-course/ta-p/493).

## How course grades work

Within the `Grades` section of CourseWorks: The `Total` course grade for each student is computed as a percentage, based on the [weighted scores](https://python-public-policy.afeld.me/en/{{school_slug}}/syllabus.html#assignments-and-evaluation). It also shows an estimated letter grade.

A few times during the semester:

1. The full gradebook is exported and run through this notebook.
1. The percentage required for all the letter grades is adjusted up or down as necessary to hit the target GPA range.
1. Those cutoffs are updated in CourseWorks.
1. The estimated letter grades potentially change.

The `min_score` column of the [new cutoffs](#new-cutoffs) shows the current minimum `Total` percentage required for each letter grade.

## Methodology

The rest of this notebook shows how the grade cutoffs are computed. **This methodology, the distribution of student percentages, and thus your estimated course grade are subject to change** up until final grades are submitted.

In [1]:
MIN_AVG_GPA = 3.2
MAX_AVG_GPA = 3.4

### Load current scores

The scores below are the total scores based on everything that has grades released thus far across both sections. See the timestamp in the filename below to know when it was updated. The grade data is anonymous for [privacy reasons](https://www.registrar.columbia.edu/content/privacy-rights-ferpa).

In [2]:
import pandas as pd

path = "/Users/afeld/Downloads/2023-10-12T0018_Grades-INAFU6504_ALL_2023_3_-_Python_for_Public_Policy.csv"
grades = pd.read_csv(path, skiprows=[1, 2])

# exclude the test student built into CourseWorks
grades = grades[grades["Student"] != "Student, Test"]

# obfuscate whose score is whose
grades = grades[["Current Score"]]
grades = grades.sort_values("Current Score").reset_index(drop=True)

grades

,Current Score
0,1.85
1,32.57
2,60.15
3,65.55
4,68.14
...,...
62,100.00
63,100.00
64,100.00
65,100.00


#### Distribution

In [3]:
import plotly.io as pio

# hack to remove Plotly MIME type that JupyterBook complains about
pio.renderers.default = "notebook_connected+pdf"

In [4]:
import plotly.express as px

fig = px.histogram(
    grades,
    x="Current Score",
    title="Distribution of the overall grades as a percentage, computed by CourseWorks",
    labels={"Current Score": "Current Score (percent)"},
)
fig.update_layout(yaxis_title_text="Number of students")
fig.show()

### Match to letter grades / GPAs

Creating the [grading notation table](https://bulletin.columbia.edu/sipa/academic-policies/grading-system-academic-progress/) in Pandas:

In [5]:
letter_grade_equivalents = pd.DataFrame(
    index=["A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D", "F"],
    data={"gpa": [4.00, 3.67, 3.33, 3.00, 2.67, 2.33, 2.00, 1.67, 1.00, 0.00]},
)

Assign starting minimum scores, roughly based on the Default Canvas Grading Scheme:

In [6]:
letter_grade_equivalents["min_score"] = [94.0, 90.0, 87.0, 84.0, 80.0, 77.0, 74.0, 70.0, 60.0, 0.0]
letter_grade_equivalents

,gpa,min_score
A,4.00,94.0
A-,3.67,90.0
B+,3.33,87.0
B,3.00,84.0
B-,2.67,80.0
C+,2.33,77.0
C,2.00,74.0
C-,1.67,70.0
D,1.00,60.0
F,0.00,0.0


### Adjust cutoffs

Raise or lower the minimum scores for each grade (not including F) until the average GPA is in the acceptable range.

In [7]:
# merge_asof() needs columns sorted ascending
orig_grade_cutoffs = letter_grade_equivalents.sort_values(by="min_score")
grade_cutoffs = orig_grade_cutoffs.copy()

grades_to_adjust = grade_cutoffs.index != "F"

adjustment = 0
STEP_SIZE = 0.1

while True:
    grade_cutoffs.loc[grades_to_adjust, "min_score"] = (
        orig_grade_cutoffs[grades_to_adjust]["min_score"] + adjustment
    )

    # make the letter grades a column so they show up in the merged DataFrame
    grade_cutoffs_with_letters = grade_cutoffs.reset_index().rename(
        columns={"index": "letter_grade"}
    )

    # find the letter grade / GPA for each student
    adjusted_grades = pd.merge_asof(
        grades,
        grade_cutoffs_with_letters,
        left_on="Current Score",
        right_on="min_score",
        direction="backward",
    )

    new_mean = adjusted_grades["gpa"].mean()
    print(f"Adjustment: {adjustment:+.1f}, Average: {new_mean:.3f}")

    # check if we've hit the target range
    if MIN_AVG_GPA <= new_mean <= MAX_AVG_GPA:
        # success
        break
    elif new_mean > MAX_AVG_GPA:
        # raise
        adjustment += STEP_SIZE
    else: # new_mean < MIN_AVG_GPA:
        # lower
        adjustment -= STEP_SIZE

Adjustment: +0.0, Average: 3.553
Adjustment: +0.1, Average: 3.553
Adjustment: +0.2, Average: 3.533
Adjustment: +0.3, Average: 3.523
Adjustment: +0.4, Average: 3.513
Adjustment: +0.5, Average: 3.513
Adjustment: +0.6, Average: 3.513
Adjustment: +0.7, Average: 3.513
Adjustment: +0.8, Average: 3.513
Adjustment: +0.9, Average: 3.508
Adjustment: +1.0, Average: 3.508
Adjustment: +1.1, Average: 3.493
Adjustment: +1.2, Average: 3.483
Adjustment: +1.3, Average: 3.483
Adjustment: +1.4, Average: 3.463
Adjustment: +1.5, Average: 3.463
Adjustment: +1.6, Average: 3.453
Adjustment: +1.7, Average: 3.453
Adjustment: +1.8, Average: 3.453
Adjustment: +1.9, Average: 3.448
Adjustment: +2.0, Average: 3.443
Adjustment: +2.1, Average: 3.438
Adjustment: +2.2, Average: 3.438
Adjustment: +2.3, Average: 3.438
Adjustment: +2.4, Average: 3.438
Adjustment: +2.5, Average: 3.438
Adjustment: +2.6, Average: 3.428
Adjustment: +2.7, Average: 3.419
Adjustment: +2.8, Average: 3.419
Adjustment: +2.9, Average: 3.419
Adjustment

Confirm the A cutoff is still achievable:

In [8]:
assert grade_cutoffs.at["A", "min_score"] <= 100

#### New cutoffs

In [9]:
grade_cutoffs.sort_values("min_score", ascending=False)

,gpa,min_score
A,4.00,97.3
A-,3.67,93.3
B+,3.33,90.3
B,3.00,87.3
B-,2.67,83.3
C+,2.33,80.3
C,2.00,77.3
C-,1.67,73.3
D,1.00,63.3
F,0.00,0.0


These will be reflected in the CourseWorks [grading scheme](https://community.canvaslms.com/t5/Canvas-Basics-Guide/What-are-grading-schemes/ta-p/41).

### Check results

Double-check the new average is in line with policy:

In [10]:
assert MIN_AVG_GPA <= new_mean <= MAX_AVG_GPA, f"{new_mean} not in acceptable range"

new_mean

3.398507462686567

In [11]:
fig = px.histogram(adjusted_grades, x="letter_grade", title="Distribution of grades")
fig.update_layout(yaxis_title_text="Number of students")
fig.show()